In [ ]:
# Import the required packages
using JuMP
using Ipopt

# Create a new optimization model using Ipopt as the solver
model = Model(Ipopt.Optimizer)

# Define the variables with bounds
@variable(model, x1 >= 0)  # x1 ≥ 0
@variable(model, x2 >= 0)  # x2 ≥ 0

# Define the objective function
@objective(model, Min, (x1 - 1)^2 + (x2 - 2)^2)

# Define the constraints
@constraint(model, x1^2 + x2^2 <= 1)  # Circle constraint

# Solve the optimization problem
optimize!(model)

# Extract the solution
x1_opt = value(x1)
x2_opt = value(x2)
optimal_value = objective_value(model)

# Display the results
println("Optimal solution:")
println("x1 = $x1_opt")
println("x2 = $x2_opt")
println("Objective value = $optimal_value")


This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9402001e+00 0.00e+00 2.64e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 